In [ ]:
import requests
import time
import pandas as pd
import re
from datetime import date, timedelta
import json
import io

In [ ]:
!gdown 'https://drive.google.com/uc?id=***************************'

In [ ]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file('/content/holodilnik-bq-376311-50cc6f8cfc32.json')

In [ ]:
date_since = (date.today()-timedelta(days=182)).strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
date_until = date.today().strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
fields = 'appmetrica_device_id,event_name,event_datetime,revenue_quantity,revenue_price,revenue_product_id,revenue_order_id,is_revenue_verified,device_manufacturer'
application_id = '*******'
url= f'https://api.appmetrica.yandex.ru/logs/v1/export/revenue_events.json?application_id={application_id}&date_since={date_since}&date_until={date_until}&fields={fields}'
header = {'Authorization': 'OAuth *************************'}
response1 = requests.get(url=url, headers=header)
k=0
while response1.status_code != 200 and k<90:
  response1 = requests.get(url=url, headers=header)
  print(response1.status_code)
  time.sleep(5)
  k+=1

In [ ]:
js1_df = response1.json()

In [ ]:
df_purchases_rec = pd.DataFrame(js1_df['data'])

In [ ]:
df_purchases_rec.sort_values('revenue_price').head(50)

In [ ]:
df_purchases_rec['full_revenue'] = df_purchases_rec['revenue_quantity'].astype('float') * df_purchases_rec['revenue_price'].astype('float')

In [ ]:
df_purchases_rec.sort_values('full_revenue').head(50)

In [ ]:
df_purchases_rec.to_gbq('holodilnik-bq-376311.appmetrica_dashboard.purchases', 
          project_id='holodilnik-bq-376311',  
          if_exists='append',
          credentials=credentials)

100%|██████████| 1/1 [00:00<00:00, 7121.06it/s]


In [ ]:
sql = """SELECT * FROM `holodilnik-bq-376311.appmetrica_dashboard.purchases`"""
all_data = pd.read_gbq(sql, credentials=credentials, dialect='standard')
all_data = all_data.drop_duplicates()

In [ ]:
all_data.to_gbq('holodilnik-bq-376311.appmetrica_dashboard.purchases', 
           project_id='holodilnik-bq-376311',  
           if_exists='replace',
           credentials=credentials)

100%|██████████| 1/1 [00:00<00:00, 2824.45it/s]
